In [12]:
from itertools import product
from collections import Counter, defaultdict

In [3]:
# p1_start = 4
# p2_start = 8

p1_start = 4
p2_start = 10

In [4]:
def die_with_n_sides(n_sides):
    while True:
        for ii in range(n_sides):
            yield ii + 1

In [5]:
def get_3(gen):
    while True:
        yield [next(gen), next(gen), next(gen)]

In [6]:
%%time

scores = [0, 0]
spaces = [p1_start, p2_start]
for ii, rolls in enumerate(get_3(die_with_n_sides(100))):
#     if ii == 8:
#         break
    player = ii % 2
    
    moves = sum(rolls)
    spaces[player] = ((spaces[player] + moves -1) % 10 ) + 1
    scores[player] += spaces[player]
#     print(f"{player=}\t {rolls=}\t {spaces[player]=}")

    if max(scores) >= 1000:
        break
        
print(scores, min(scores) * (ii + 1) * 3)
    

[926, 1005] 855624
Wall time: 0 ns


# part 2


* keep track of the number of universes in which each possible score difference is achieved
* at each turn, in each universe, 9 universes are generated, where the given player rolls `[1, 1, 1], [1, 1, 2], ... [3, 3, 3]`
* for each player, keep track of the number of universes in which they are on a given space
* for each player, keep track of the number of universes in which they have each score

###  NOPE. each game ends when ANY player's score is geq 21. therefore, keep track of all PAIRS of scores `(p1_score, p2_score)`

### need to keep track of number of universes at specific spaces?
### need to keep track of number of universes with specific score pairs?


### thoughts
* new score delta depends only on what space a player lands on
* a 'turn' can be considered to happen simultaneously across all universes
* a 'turn' maps a count of universes with scores to a new count of universes with scores
* ~ 100 possible space pairs
* ~ 400 possible score pairs
* therefore, ~40k possible (space, score) tuples
* is there a way to reduce this? do these have to be kept track of together?
    * space pair counts are sufficient to generate next iteration of space pair counts
        * each pair `{(p1 space, p2 space): count}` --> turn p1 --> `{(p1 space + move_1, p2 space): += count * mult_2, (p1 space + move_2, p2 space): += count * mult_2}, (p1 space + move_3, p2 space): += count * mult_3}`
    * 


### algorithm

* keep track of the count of universes with each score pair
* for each turn:
    * for each score count:
        * increase the number of universes 

In [14]:
moves_count = Counter(sum(items) for items in product(*[[1, 2, 3]]*3))
curr_space = 9

lookup_mults = dict()
for curr_space in range(1, 10 + 1):
    spaces_count = {((curr_space + move - 1) % 10 + 1): count for move, count in moves_count.items()}
    lookup_mults[curr_space] = spaces_count
# moves_count, spaces_count
lookup_mults

{1: {4: 1, 5: 3, 6: 6, 7: 7, 8: 6, 9: 3, 10: 1},
 2: {5: 1, 6: 3, 7: 6, 8: 7, 9: 6, 10: 3, 1: 1},
 3: {6: 1, 7: 3, 8: 6, 9: 7, 10: 6, 1: 3, 2: 1},
 4: {7: 1, 8: 3, 9: 6, 10: 7, 1: 6, 2: 3, 3: 1},
 5: {8: 1, 9: 3, 10: 6, 1: 7, 2: 6, 3: 3, 4: 1},
 6: {9: 1, 10: 3, 1: 6, 2: 7, 3: 6, 4: 3, 5: 1},
 7: {10: 1, 1: 3, 2: 6, 3: 7, 4: 6, 5: 3, 6: 1},
 8: {1: 1, 2: 3, 3: 6, 4: 7, 5: 6, 6: 3, 7: 1},
 9: {2: 1, 3: 3, 4: 6, 5: 7, 6: 6, 7: 3, 8: 1},
 10: {3: 1, 4: 3, 5: 6, 6: 7, 7: 6, 8: 3, 9: 1}}

In [23]:
def update(universe_counts, whose_turn):
    """ 
    * given a universe of counts, generate a new universe with updated counts
    * 'end' games with max score >= 21
    
    universe_counts is a dict of {(p1 space, p2 space, p1 score, p2 score): count} 
    """
    updated = defaultdict(int)
    p1_wins = 0
    p2_wins = 0
    for (p1_space, p2_space, p1_score, p2_score), count in universe_counts.items():
        # TODO: skip the won games
#         print((p1_space, p2_space, p1_score, p2_score), count)
        if p1_score >= 21:
            p1_wins += count
            continue
        elif p2_score >= 21:
            p2_wins += count
            continue
        else:
            if whose_turn == 0:
                for space, mult in lookup_mults[p1_space].items():
                    updated[(space, p2_space, p1_score + space, p2_score)] += mult * count
            else:
                for space, mult in lookup_mults[p2_space].items():
                    updated[(p1_space, space, p1_score, p2_score + space)] += mult * count
    return updated, p1_wins, p2_wins

uni = {(1, 2, 16, 0): 1}
u1, _, _ = update(uni, 0)
print(u1)
update(u1, 0)

defaultdict(<class 'int'>, {(4, 2, 20, 0): 1, (5, 2, 21, 0): 3, (6, 2, 22, 0): 6, (7, 2, 23, 0): 7, (8, 2, 24, 0): 6, (9, 2, 25, 0): 3, (10, 2, 26, 0): 1})


(defaultdict(int,
             {(7, 2, 27, 0): 1,
              (8, 2, 28, 0): 3,
              (9, 2, 29, 0): 6,
              (10, 2, 30, 0): 7,
              (1, 2, 21, 0): 6,
              (2, 2, 22, 0): 3,
              (3, 2, 23, 0): 1}),
 26,
 0)

In [31]:
%%time

# uni_prev = {(4, 8, 0, 0): 1}
uni_prev = {(4, 10, 0, 0): 1}

wins_p1, wins_p2 = 0, 0
for ii in range(24):
    uni, wp1, wp2 = update(uni_prev, ii % 2)
    wins_p1 += wp1
    wins_p2 += wp2
    if uni == uni_prev:
        print(f"no changes at turn {ii}")
        break
    uni_prev = uni
    
wins_p1, wins_p2, max(wins_p1, wins_p2)

no changes at turn 20
Wall time: 187 ms


(187451244607486, 183752194019471, 187451244607486)